# SVM 모델 GCS log 성능 도출 결과

### 필요한 라이브러리 import

In [ ]:
# import all_imports

#!/usr/bin/env python3

from __future__ import print_function, division
import numpy as np
import pandas as pd
from scipy.interpolate import griddata, interp1d

import matplotlib
import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.style.use('default')
#plotting lib, but just adding makes the matplotlib plots look better
import seaborn 

import matplotlib as mpl
mpl.style.use('default') #seaborn, default, seaborn-paper

import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
# from sklearn import svm
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as ms

import joblib
from sklearn.metrics import matthews_corrcoef, confusion_matrix, multilabel_confusion_matrix

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
np.set_printoptions(precision=6, suppress=True)

import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/swcon_capstone

/content/drive/My Drive/Colab Notebooks/swcon_capstone


In [ ]:
# Try to import PPRZ_DATA (thinking that you have already installed it...)

from pprz_data.pprz_data import DATA

#If that fails : comment the above line and uncomment the below ones to use the cloned git submodule of PPRZ_DATA
# import sys
# sys.path.append("..")         
# from pprz_data.pprz_data import DATA

In [ ]:
def add_time_history_1(X,y,n_step=3):
    '''X[0] = Ax0Ay0Az0Gx0Gy0Gz0Ax1Ay1Az1Gx1Gy1Gz1...'''
    time_len= X.shape[0] #-n_step
    column_len = X.shape[1]

    xx=np.zeros((time_len,column_len*n_step))

    for i in range(n_step, time_len):
        for j in range(n_step):
            xx[i,j*column_len:(j+1)*column_len] = X[i-n_step+j]
            
    xx = xx[n_step:X.shape[0],:]
    yy = y[n_step:]
    
    return xx,yy

def add_time_history_2(X,y,n_step=3):
    '''X[0] = Ax0Ax1Ay0Ay1Az0Az1Gx0Gx1Gy0Gy1Gz0Gz1....'''
    time_len = X.shape[0]
    column_len = X.shape[1]

    xx = np.zeros((time_len,column_len * n_step))

    for i in range(n_step,time_len):
        for j in range(column_len):
            xx[i,j*n_step:(j+1)*n_step] = X[(i+0-n_step):(i+0),j]

    xx = xx[n_step:X.shape[0],:]
    yy = y[n_step:]

    return xx,yy

def assign_fault(df):
    df = df.assign(fault=0)
    cond1 = (df['add1'] > 0.005) | (df['add1'] < -0.005)
    cond2 = (df['add2'] > 0.005) | (df['add2'] < -0.005)
    cond3 = (df['m1'] < 1.0) | (df['m2'] < 1.0)
    cond = cond1 | cond2 | cond3
    df.loc[cond,'fault'] = 1
#     cond4 = (df['mode'] == 2.0)
#     df = df[cond4]
    return df

# Preparing to do a simple binary classification

In [ ]:
class Time():
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
def train_SVM(X_train, X_test, y_train, y_test, features=['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz', 'C1', 'C2'], targets=['nominal', 'faulty']):
    # Create the Feature and Label List
    columns=features #['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz', 'C1', 'C2']
    """
    if time==None:
        flight = df.copy()
    else:
        flight = df[time.start:time.end].copy()                                                                                                                     
    X_pre = flight[columns].values # Features          
    y = flight.fault.values   # Labels
    X_sd,y_sd = add_time_history_1(X_pre,y,n_step= 20)

    X_train, X_test, y_train, y_test = ms.train_test_split(X_sd, y_sd, test_size=0.2, random_state=42, stratify=y_sd)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    """
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': ['auto'], 'C': [1, 5, 10, 20]}]
    score = 'f1_micro'
    # score = 'accuracy'

    print("# Tuning hyper-parameters for %s" % score)
    print()

    # cv : For integer/None inputs, if the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used.
    clf = GridSearchCV(SVC(), tuned_parameters, scoring=score)  # scoring='%s_micro' % score)
    clf.fit(X_train, y_train)


    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    time1 = time.time()
    y_true, y_pred = y_test, clf.predict(X_test)
    time2 = time.time()
    print("Calculate Time:", time2-time1)
    #target_names = targets
    print(classification_report(y_true, y_pred)) #, target_names=target_names
    print('Matthews Correlation Coefficient : ',  matthews_corrcoef(y_true, y_pred))
    #return clf, scaler
"""
def predict_SVM(model, scaler, df, time=None, features=['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz', 'C1', 'C2'], targets=['nominal', 'faulty']):
    # Create the Feature and Label List
    columns=features
    if time==None:
        flight = df.copy()
    else:
        flight = df[time.start:time.end].copy()                                                                                                                    
    X_pre = flight[columns].values # Features          
    y = flight.fault.values   # Labels
    X_sd,y_sd = add_time_history_1(X_pre,y,n_step= 20)

    X_sd_scaled = scaler.transform(X_sd)
    X_sd_scaled_reduced = X_sd_scaled[::20,:].copy()
    y_sd_reduced = y_sd[::20].copy()
    y_sd_pred = model.predict(X_sd_scaled_reduced)

#     target_names = ['nominal', 'R0.3', 'L0.9', 'L0.8','L0.7', 'L0.6', 'L0.5', 'L0.4', 'L0.3']
    print(classification_report(y_sd_reduced, y_sd_pred, target_names=target_names))
    return y_sd_reduced, y_sd_pred
"""

"\ndef predict_SVM(model, scaler, df, time=None, features=['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz', 'C1', 'C2'], targets=['nominal', 'faulty']):\n    # Create the Feature and Label List\n    columns=features\n    if time==None:\n        flight = df.copy()\n    else:\n        flight = df[time.start:time.end].copy()                                                                                                                    \n    X_pre = flight[columns].values # Features          \n    y = flight.fault.values   # Labels\n    X_sd,y_sd = add_time_history_1(X_pre,y,n_step= 20)\n\n    X_sd_scaled = scaler.transform(X_sd)\n    X_sd_scaled_reduced = X_sd_scaled[::20,:].copy()\n    y_sd_reduced = y_sd[::20].copy()\n    y_sd_pred = model.predict(X_sd_scaled_reduced)\n\n#     target_names = ['nominal', 'R0.3', 'L0.9', 'L0.8','L0.7', 'L0.6', 'L0.5', 'L0.4', 'L0.3']\n    print(classification_report(y_sd_reduced, y_sd_pred, target_names=target_names))\n    return y_sd_reduced, y_sd_pred\n"

### Load all of the GCS data

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/swcon_capstone/fault_detection


In [ ]:
X_train_bin = np.load("X_train_bin.npy")
y_train_bin = np.load("y_train_bin.npy")
X_test_bin = np.load("X_test_bin.npy")
y_test_bin = np.load("y_test_bin.npy")

X_train_three = np.load("X_train_three.npy")
y_train_three = np.load("y_train_three.npy")
X_test_three = np.load("X_test_three.npy")
y_test_three = np.load("y_test_three.npy")

X_train_nine = np.load("X_train_nine.npy")
y_train_nine = np.load("y_train_nine.npy")
X_test_nine = np.load("X_test_nine.npy")
y_test_nine = np.load("y_test_nine.npy")

In [ ]:
print(y_train_bin.shape)
print(y_test_bin.shape)
print(y_train_three.shape)
print(y_test_three.shape)
print(y_train_nine.shape)
print(y_test_nine.shape)

(171384,)
(42846,)
(50192,)
(12548,)
(23696,)
(5924,)


#### binary classification

In [ ]:
train_SVM(X_train_bin[161000:], X_test_bin[40000:], y_train_bin[161000:], y_test_bin[40000:], targets=['Fault_Telemetry'])

# Tuning hyper-parameters for f1_micro

Best parameters set found on development set:

{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}

Grid scores on development set:

0.881 (+/-0.002) for {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.886 (+/-0.002) for {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
0.887 (+/-0.008) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.883 (+/-0.010) for {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Calculate Time: 2.835103988647461
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      2503
           1       0.67      0.14      0.23       343

    accuracy                           0.89      2846
   macro avg       0.78      0.56      0.58      2846
weighted avg       0.87      0.89      0.85      2846

Matthews Correlation Coefficient :  0.2687052843679111


#### 3-class multi class classification

In [ ]:
train_SVM(X_train_three[47000:], X_test_three[11900:], y_train_three[47000:], y_test_three[11900:], targets=['Fault_Telemetry'])

# Tuning hyper-parameters for f1_micro

Best parameters set found on development set:

{'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}

Grid scores on development set:

0.859 (+/-0.023) for {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.867 (+/-0.023) for {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
0.867 (+/-0.021) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.870 (+/-0.023) for {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       385
           1       0.76      0.72      0.74       102
           2       0.96      0.94      0.95       161

    accuracy                           0.90       648
   macro avg       0.87      0.86      0.87       648
weighted avg       0.90      0.90      0.90       648

Matthews Correlation Coefficient :  0.8140491092045532


#### 9-class multi class classification

In [ ]:
train_SVM(X_train_nine[20000:], X_test_nine[5000:], y_train_nine[20000:], y_test_nine[5000:], targets=['Fault_Telemetry'])

# Tuning hyper-parameters for f1_micro

Best parameters set found on development set:

{'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}

Grid scores on development set:

0.627 (+/-0.017) for {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.691 (+/-0.024) for {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
0.703 (+/-0.028) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.707 (+/-0.022) for {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       477
           1       0.68      0.44      0.53       108
           2       0.60      0.25      0.36        59
           3       0.46      0.37      0.41        46
           4       0.55      0.48      0.52        33
           5       0.65      0.45      0.53        29
           6       0.74      0.68      0.71        34
       

## Train - test - validate and report  using only $12^{th}$ of July flight data

In [ ]:
# time class to define the portion of the flight to be used during training
time = Time(500,1400)
# Lots of things going on inside train_SVM() function, so please check out the above definition for more info
clf, scaler = train_SVM(df_12, time, targets = ['Nominal', 'Faulty'] )

# Tuning hyper-parameters for f1_micro

Best parameters set found on development set:

{'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}

Grid scores on development set:

0.978 (+/-0.006) for {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.983 (+/-0.008) for {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
0.983 (+/-0.008) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.981 (+/-0.007) for {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

     Nominal       0.99      0.99      0.99       843
      Faulty       0.99      0.99      0.99       956

    accuracy                           0.99      1799
   macro avg       0.99      0.99      0.99      1799
weighted avg       0.99      0.99      0.99      1799

Matthews Correlation Coefficient :  0.9799400202934504
